In [10]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

import numpy as np
import pandas as pd

from quantfreedom.base import backtest_df_only
from quantfreedom.nb.simulate import simulate_up_to_6_nb
from quantfreedom.evaluators.evaluators import eval_is_below
from quantfreedom.indicators.talib_ind import from_talib
from quantfreedom.enums.enums import (
    LeverageMode,
    SizeType,
    OrderType,
    SL_BE_or_Trail_BasedOn,
)
from quantfreedom.testing import *
from quantfreedom.plotting.strat_dash import strat_dashboard

np.set_printoptions(formatter={"float_kind": "{:.2f}".format})
pd.options.display.float_format = "{:,.2f}".format

prices = pd.read_hdf("../data/bigtest.hd5")

rsi_ind = from_talib(
    func_name="rsi",
    prices=prices,
    cart_product=False,
    combos=False,
    timeperiod=[15, 30],
)
entries = eval_is_below(
    want_to_evaluate=rsi_ind,
    user_args=[30, 40, 50],
    cand_ohlc="close",
)

In [2]:
entries

symbol              BTCUSDT                                    EOSUSD         \
RSI_timeperiod           15                   30                   15          
RSI_is_below             30     40     50     30     40     50     30     40   
open_time                                                                      
2020-10-01 00:00:00   False  False  False  False  False  False  False  False   
2020-10-01 00:30:00   False  False  False  False  False  False  False  False   
2020-10-01 01:00:00   False  False  False  False  False  False  False  False   
2020-10-01 01:30:00   False  False  False  False  False  False  False  False   
2020-10-01 02:00:00   False  False  False  False  False  False  False  False   
...                     ...    ...    ...    ...    ...    ...    ...    ...   
2021-04-14 21:30:00   False  False   True  False  False   True  False  False   
2021-04-14 22:00:00   False  False   True  False  False   True  False  False   
2021-04-14 22:30:00   False  False   True  False  False   True  False  False   
2021-04-14 23:00:00   False  False   True  False  False   True  False  False   
2021-04-14 23:30:00   False  False   True  False  False   True  False  False   

symbol                             ... LTCUSDT                      XRPUSD  \
RSI_timeperiod                 30  ...      15     30                   15   
RSI_is_below            50     30  ...      50     30     40     50     30   
open_time                          ...                                       
2020-10-01 00:00:00  False  False  ...   False  False  False  False  False   
2020-10-01 00:30:00  False  False  ...   False  False  False  False  False   
2020-10-01 01:00:00  False  False  ...   False  False  False  False  False   
2020-10-01 01:30:00  False  False  ...   False  False  False  False  False   
2020-10-01 02:00:00  False  False  ...   False  False  False  False  False   
...                    ...    ...  ...     ...    ...    ...    ...    ...   
2021-04-14 21:30:00  False  False  ...   False  False  False  False  False   
2021-04-14 22:00:00  False  False  ...   False  False  False  False  False   
2021-04-14 22:30:00  False  False  ...   False  False  False  False  False   
2021-04-14 23:00:00  False  False  ...   False  False  False  False  False   
2021-04-14 23:30:00  False  False  ...   False  False  False  False  False   

symbol                                                  
RSI_timeperiod                        30                
RSI_is_below            40     50     30     40     50  
open_time                                               
2020-10-01 00:00:00  False  False  False  False  False  
2020-10-01 00:30:00  False  False  False  False  False  
2020-10-01 01:00:00  False  False  False  False  False  
2020-10-01 01:30:00  False  False  False  False  False  
2020-10-01 02:00:00  False  False  False  False  False  
...                    ...    ...    ...    ...    ...  
2021-04-14 21:30:00  False  False  False  False  False  
2021-04-14 22:00:00  False  False  False  False  False  
2021-04-14 22:30:00  False  False  False  False  False  
2021-04-14 23:00:00  False  False  False  False  False  
2021-04-14 23:30:00  False  False  False  False  False  

[9408 rows x 30 columns]

In [3]:
strat_results_df, settings_results_df = backtest_df_only(
    prices=prices,
    entries=entries,
    equity=1000.0,
    fee_pct=0.06,
    mmr=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=np.arange(3, 7.1, 1),
    risk_rewards=np.arange(3, 11.1, .5),
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_pcts_init=np.arange(2, 9.1, .5),
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_trail_by_pct=np.arange(2, 9.1, .5),
    tsl_when_pct_from_avg_entry=np.arange(2, 9.1, 1),
    gains_pct_filter=150,
)

In [4]:
strat_results_df

,symbol,entries_col,or_set,total_trades,gains_pct,win_rate,to_the_upside,total_pnl,ending_eq
322136,EOSUSD,8,123937,57.00,242.27,45.61,0.89,"2,422.68","3,422.68"
356973,EOSUSD,11,123937,57.00,243.27,43.86,0.89,"2,432.74","3,432.74"
344719,EOSUSD,11,62856,56.00,152.87,44.64,0.89,"1,528.71","2,528.71"
344959,EOSUSD,11,64656,54.00,150.01,48.15,0.88,"1,500.08","2,500.08"
19536,BTCUSDT,1,62834,36.00,167.91,55.56,0.88,"1,679.11","2,679.11"
...,...,...,...,...,...,...,...,...,...
402559,ETHUSD,14,118778,95.00,157.28,34.74,0.00,"1,572.84","2,572.84"
469964,ETHUSD,17,135636,194.00,262.62,30.41,0.00,"2,626.20","3,626.20"
454147,ETHUSD,17,108307,280.00,185.16,17.86,0.00,"1,851.58","2,851.58"
400646,ETHUSD,14,114141,193.00,226.29,31.09,0.00,"2,262.91","3,262.91"


In [5]:
settings_results_df.iloc[:,1251]

symbol                           BTCUSDT
entries_col                            0
max_equity_risk_pct                 6.00
risk_rewards                        8.00
size_pct                            1.00
tsl_based_on                   low_price
tsl_pcts_init                       6.00
tsl_trail_by_pct                    7.50
tsl_when_pct_from_avg_entry         2.00
Name: 1251, dtype: object

In [6]:
order_records = simulate_up_to_6_nb(
    prices=prices['BTCUSDT'].values,
    entries=entries.iloc[:,[5]].values,
    equity=1000.0,
    fee_pct=0.06,
    mmr=0.5,
    lev_mode=LeverageMode.LeastFreeCashUsed,
    size_type=SizeType.RiskPercentOfAccount,
    order_type=OrderType.LongEntry,
    max_equity_risk_pct=5,
    risk_rewards=2,
    size_pct=1.0,
    # sl_pcts=np.arange(1,3.1,1),
    tsl_true_or_false=True,
    tsl_based_on=SL_BE_or_Trail_BasedOn.low_price,
    tsl_pcts_init=5,
    tsl_trail_by_pct=3,
    tsl_when_pct_from_avg_entry=3,
    # gains_pct_filter=100,
)

In [7]:
entries[[("BTCUSDT", 30, 50)]]

symbol,BTCUSDT
RSI_timeperiod,30
RSI_is_below,50
open_time,
2020-10-01 00:00:00,False
2020-10-01 00:30:00,False
2020-10-01 01:00:00,False
2020-10-01 01:30:00,False
2020-10-01 02:00:00,False
...,...
2021-04-14 21:30:00,True


In [8]:
rsi_ind[[('BTCUSDT', 30)]]

symbol,BTCUSDT
RSI_timeperiod,30
open_time,
2020-10-01 00:00:00,NaN
2020-10-01 00:30:00,NaN
2020-10-01 01:00:00,NaN
2020-10-01 01:30:00,NaN
2020-10-01 02:00:00,NaN
...,...
2021-04-14 21:30:00,47.78
2021-04-14 22:00:00,46.99


In [9]:
indicator_dict = {
    "indicator1": {
        "values1": rsi_ind[[('BTCUSDT', 30)]],
        "entries1": entries[[("BTCUSDT", 30, 50)]],
    },
}
strat_dashboard(
    indicator_dict=indicator_dict,
    prices=prices.BTCUSDT,
    order_records=order_records,
)

Dash is running on http://127.0.0.1:3003/

Dash app running on http://127.0.0.1:3003/
